<a href="https://colab.research.google.com/github/molinamarriott/challenges/blob/main/WebScrappingSky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver-manager

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/ubuntu 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import requests
from bs4 import BeautifulSoup

urls = ['https://www.skysports.com/premier-league-results/2022-23',
        'https://www.skysports.com/premier-league-results/2021-22',
        "https://www.skysports.com/premier-league-results/2020-21"]

links = []

for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  matches = soup.find_all('div', {'class': 'fixres__item'})

  for match in matches:
    link_match = str(match.find('a')["href"])
    links.append(link_match)

In [32]:
def visit_url(url):
  import requests
  from bs4 import BeautifulSoup
  from selenium import webdriver
  import pandas as pd


  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
# Navigate to the page
  driver.get(url)

# Wait for the page to load
  driver.implicitly_wait(1000)

# Use BeautifulSoup to parse the page source
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  return soup

In [33]:
def get_variables(url_bruto):
  url_bruto[:-6] + 'teams/' + url_bruto[-6:]
  # Create a new instance of the Chrome driver
  url = url_bruto[:-6] + 'stats/' + url_bruto[-6:]

  soup = visit_url(url)

  table = soup.find("div",{"class":"sdc-site-match-stats__inner"})

  stats = table.find_all("h5", {'class':"sdc-site-match-stats__name"})
  
  variable_names = ["Home","Away"]
  values = []
  
  local = table.find("span", {'class':"sdc-site-match-stats__team-name"}).get_text(strip=True)
  away = table.find("span", {'class':"sdc-site-match-stats__team-name sdc-site-match-stats__team-name--away"}).get_text(strip=True)

  values.append(local)
  values.append(away)

  for stat in stats:
    variable_names.append(f'Home {stat.get_text(strip=True)}')
    variable_names.append(f'Away {stat.get_text(strip=True)}')

  results = soup.find_all("span", {'class':"sdc-site-match-stats__val"})
  for result in results:
    values.append(result.get_text(strip=True))

  fecha = soup.find("time").get_text(strip=True)
  variable_names.append("Fecha")
  values.append(fecha)

  url2 = url.replace("stats", "teams")
  soup = visit_url(url2)

  referee = soup.find("dd", {"class":'sdc-site-team-lineup__officials-name', 'data-officials-role':"Referee"}).get_text(strip=True)
  variable_names.append("Referee")
  values.append(referee)

  df = pd.DataFrame([values], columns=variable_names)

  return df

In [36]:
n = 1
for link in links:
  if n == 1:
    df = get_variables(link)
  else:
    df = df.append(get_variables(link))
  n += 1

df

,Home,Away,Home Possession %,Away Possession %,Home Total Shots,Away Total Shots,Home On Target,Away On Target,Home Off Target,Away Off Target,...,Home Fouls Committed,Away Fouls Committed,Home Fouls Won,Away Fouls Won,Home Yellow Cards,Away Yellow Cards,Home Red Cards,Away Red Cards,Fecha,Referee
0,Chelsea,C Palace,62.5,37.5,15,10,5,5,5,2,...,10,17,17,10,2,5,0,0,"2:00pm, Sunday 15th January 2023.",Peter Bankes
0,Newcastle,Fulham,62.3,37.7,20,5,5,0,7,4,...,6,14,12,5,1,3,0,0,"2:00pm, Sunday 15th January 2023.",Robert Jones
0,Tottenham,Arsenal,50.6,49.4,17,14,7,5,4,5,...,16,15,13,16,4,2,0,0,"4:30pm, Sunday 15th January 2023.",Craig Pawson
0,Man Utd,Man City,29.4,70.6,8,5,4,1,1,2,...,15,5,5,15,3,0,0,0,"12:30pm, Saturday 14th January 2023.",Stuart Attwell
0,Brighton,Liverpool,60.8,39.2,16,6,9,2,3,3,...,9,15,14,9,1,3,0,0,"3:00pm, Saturday 14th January 2023.",Darren England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,A Villa,Newcastle,59.2,40.8,14,7,5,1,6,4,...,9,18,18,9,0,3,0,0,"8:00pm, Saturday 23rd January 2021.",Simon Hooper
0,Liverpool,Burnley,71.8,28.2,27,6,6,4,10,2,...,11,8,7,10,2,1,0,0,"8:00pm, Thursday 21st January 2021.",Mike Dean
0,Man City,A Villa,69.1,30.9,28,11,9,4,7,4,...,10,9,8,10,0,2,0,0,"6:00pm, Wednesday 20th January 2021.",Jonathan Moss
0,Fulham,Man Utd,38.5,61.5,12,15,5,5,4,6,...,6,10,10,6,3,1,0,0,"8:15pm, Wednesday 20th January 2021.",Martin Atkinson


In [37]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/datos_premier.csv")